In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
! pip install transformers trl peft accelerate datasets bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.8 MB/s eta 0:00:00:00:0100:01


In [3]:
from huggingface_hub import notebook_login

notebook_login()  # same as before

In [4]:
import os
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from peft import AutoPeftModelForCausalLM, LoraConfig
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments

from trl import SFTTrainer
from trl.trainer import ConstantLengthDataset

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:

def chars_token_ratio(dataset, tokenizer, nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """
    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        text = prepare_sample_text(example)
        total_characters += len(text)
        if tokenizer.is_fast:
            total_tokens += len(tokenizer(text).tokens())
        else:
            total_tokens += len(tokenizer.tokenize(text))

    return total_characters / total_tokens


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


In [6]:
def prepare_sample_text(example):
    """Prepare the text from a sample of the dataset."""
    text = f"{example['prompt']}\n {example['response']}"
    return text


def create_datasets(tokenizer):
    dataset = load_dataset(
        "Dahoas/full-hh-rlhf",
        split = "test",
        use_auth_token=True
    )
    dataset = dataset.train_test_split(test_size=0.005, seed=None)
    train_data = dataset["train"]
    valid_data = dataset["test"]
    print(f"Size of the train set: {len(train_data)}. Size of the validation set: {len(valid_data)}")

    chars_per_token = chars_token_ratio(train_data, tokenizer)
    print(f"The character to token ratio of the dataset is: {chars_per_token:.2f}")

    train_dataset = ConstantLengthDataset(
        tokenizer,
        train_data,
        formatting_func=prepare_sample_text,
        infinite=True,
        seq_length=256,
        chars_per_token=chars_per_token,
    )
    valid_dataset = ConstantLengthDataset(
        tokenizer,
        valid_data,
        formatting_func=prepare_sample_text,
        infinite=False,
        seq_length=256,
        chars_per_token=chars_per_token,
    )
    return train_dataset, valid_dataset

In [12]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

base_model = AutoModelForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    quantization_config=bnb_config,
    device_map={"": 0},
    trust_remote_code=True,
    use_auth_token=True,
)
base_model.config.use_cache = False

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["c_attn"],
    bias="none",
    task_type="CAUSAL_LM",
)

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [13]:
tokenizer = AutoTokenizer.from_pretrained("decapoda-research/llama-7b-hf", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training


training_args = TrainingArguments(
    output_dir="llama7b",
    per_device_train_batch_size=10,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=10,
    learning_rate=2e-4,
    logging_steps=10,
    max_steps=105,
    report_to="tensorboard",
    save_steps=100,
    lr_scheduler_type="cosine",
    warmup_steps=1,
    optim="paged_adamw_32bit",
    fp16=True,
    remove_unused_columns=False,
    run_name="llama7b",
)

train_dataset, eval_dataset = create_datasets(tokenizer)

ValueError: Tokenizer class LLaMATokenizer does not exist or is not currently imported.

In [11]:
print(base_model)

In [9]:
trainer = SFTTrainer(
    model=base_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    packing=True,
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_args,
)
trainer.train()
trainer.save_model("sft_santacoder1b")

output_dir = os.path.join("sft_santacoder1b", "final_checkpoint")
trainer.model.save_pretrained(output_dir)

# Free memory for merging weights
del base_model
torch.cuda.empty_cache()

model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.float16)
model = model.merge_and_unload()

output_merged_dir = os.path.join("sft_santacoder1b", "final_merged_checkpoint")
model.save_pretrained(output_merged_dir, safe_serialization=True)

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


ValueError: Target modules ['c_attn'] not found in the base model. Please check the target modules and try again.

In [ ]:
! cp -r /content/sft_santacoder1b /content/drive/MyDrive


In [10]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

base_model = AutoModelForCausalLM.from_pretrained(
    "TabbyML/SantaCoder-1B",
    quantization_config=bnb_config,
    device_map={"": 0},
    trust_remote_code=True,
    use_auth_token=True,
)
base_model.config.use_cache = False

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:479: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [11]:
print(base_model)


# DPO

In [12]:
# 0. imports
import os
from dataclasses import dataclass, field
from typing import Dict, Optional

import torch
from datasets import Dataset, load_dataset
from peft import AutoPeftModelForCausalLM, LoraConfig
from transformers import AutoTokenizer, HfArgumentParser, TrainingArguments

from trl import DPOTrainer

In [14]:
def dpo_data(train_or_val):

    dataset = load_dataset(
        "Dahoas/full-hh-rlhf",
        split = "train",
        use_auth_token=True
    )

    original_columns = dataset.column_names

    def return_prompt_and_responses(samples):
        return {
            "prompt": [prompt for prompt in samples["prompt"]],
            "chosen": samples["chosen"],
            "rejected": samples["rejected"],
        }

    return dataset.map(
        return_prompt_and_responses,
        batched=True,
        remove_columns=original_columns,
    )

if __name__ == "__main__":

    # 1. load a pretrained model
    model = AutoPeftModelForCausalLM.from_pretrained(
        "/kaggle/working/sft_santacoder1b",
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        load_in_4bit=True,
    )
    model.config.use_cache = False

    model_ref = AutoPeftModelForCausalLM.from_pretrained(
        "/kaggle/working/sft_santacoder1b",
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        load_in_4bit=True,
    )
    tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/sft_santacoder1b")
    tokenizer.pad_token = tokenizer.eos_token

    train_dataset = dpo_data("train")
    train_dataset = train_dataset.filter(
        lambda x: len(x["prompt"]) + len(x["chosen"]) <= 256
        and len(x["prompt"]) + len(x["rejected"]) <= 256
    )
    eval_dataset = dpo_data("val")
    eval_dataset = eval_dataset.filter(
        lambda x: len(x["prompt"]) + len(x["chosen"]) <= 256
        and len(x["prompt"]) + len(x["rejected"]) <= 256
    )
    training_args = TrainingArguments(
        per_device_train_batch_size=2,
        max_steps=505,
        logging_steps=10,
        save_steps=500,
        gradient_accumulation_steps=4,
        gradient_checkpointing=True,
        learning_rate=2e-4,
        evaluation_strategy="steps",
        eval_steps=100,
        output_dir="dpo_santacoder1b",
        report_to="tensorboard",
        lr_scheduler_type="cosine",
        warmup_steps=2,
        optim="paged_adamw_32bit",
        fp16=True,
        remove_unused_columns=False,
        run_name="dpo_llama2",
    )

    peft_config = LoraConfig(
        r=8,
        lora_alpha=16,
        lora_dropout=0.05,
        target_modules=["c_attn", "c_proj"],
        bias="none",
        task_type="CAUSAL_LM",
    )
    dpo_trainer = DPOTrainer(
        model,
        model_ref,
        args=training_args,
        beta=0.1,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        peft_config=peft_config,
        max_prompt_length=128,
        max_length=256,
    )

    # 6. train
    dpo_trainer.train()
    dpo_trainer.save_model("dpo_santacoder1b")

    # 7. save
    output_dir = os.path.join("dpo_santacoder1b", "final_checkpoint")
    dpo_trainer.model.save_pretrained(output_dir)

  0%|          | 0/113 [00:00<?, ?ba/s]

  0%|          | 0/113 [00:00<?, ?ba/s]

  0%|          | 0/113 [00:00<?, ?ba/s]

  0%|          | 0/113 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen
100,0.684800,0.620088,-0.928724,-1.192336,0.653653,0.263612,-71.984299,-78.585617,-1.122388,-1.229258
200,0.649000,0.577441,-0.176761,-0.596827,0.704724,0.420066,-66.029213,-71.065987,-1.196013,-1.300713
300,0.618200,0.537989,-0.757476,-1.349462,0.729781,0.591987,-73.555565,-76.873138,-1.203135,-1.297478
400,0.569900,0.525641,-0.882802,-1.499920,0.741464,0.617118,-75.060150,-78.126389,-1.183675,-1.277893
500,0.532600,0.523378,-0.789281,-1.403192,0.743318,0.613910,-74.092857,-77.191193,-1.180364,-1.275136


In [15]:

from transformers import AutoModelForCausalLM
from peft import PeftModel
model = AutoModelForCausalLM.from_pretrained(
        "/kaggle/working/sft_santacoder1b/final_merged_checkpoint", return_dict=True, torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(model, "/kaggle/working/sft_santacoder1b/final_checkpoint")
model.eval()
model = model.merge_and_unload()

model.save_pretrained("/kaggle/working/sft_santacoder1b/final_merged_checkpoint")
tokenizer.save_pretrained("/kaggle/working/sft_santacoder1b/final_merged_checkpoint")
model.push_to_hub("dpo-santacoder1b")
tokenizer.push_to_hub("dpo-santacoder1b")

RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-65129b9c-1e9bbc83548b796136f96bf8;df73b086-13db-4818-8db1-1971ba7cfc98)

Repository Not Found for url: https://huggingface.co/api/models/dpo-santacoder1b.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.

# Inference

In [6]:
from transformers import pipeline
pipe_dpo = pipeline("text-generation", "Vasanth/dpo-santacoder1b", max_length=256, top_p=0.9, device="cuda", no_repeat_ngram_size=3)

In [10]:
pipe_dpo("Human: Should you buy a case to protect your cell phone? Assistant:")

Setting `pad_token_id` to `eos_token_id`:49152 for open-end generation.


[{'generated_text': 'Human: Should you buy a case to protect your cell phone? Assistant: Yes, I would like to help you protect your phone from the infection of the cell phone.\nHuman: How do you protect the phone from infection?\nAssistant: I would love to help protect your mobile phone from being infected by the injury of the insects.\nAnswer: I think you can help protect the mobile phone by protecting the phone with a lock.\n\nHuman 2:\nHuman answer:\nAnswer 1: Yes\nAnswer2: No\n\nAnswer1:\n\nI would like you to help prevent the invasion of the virus from the victim.\nI will help you to protect the vaccine from the disease of the disease.\nYou can help me to protect my vaccine by protect the disease with a virus.\nIf you are sure that you are protecting my vaccines, you can also help protect my medical care by protect my health care.\n'}]

In [11]:
from transformers import pipeline
pipe = pipeline("text-generation", "TabbyML/SantaCoder-1B", max_length=256, top_p=0.9, device="cuda", no_repeat_ngram_size=3)

    

In [12]:
pipe("Should you buy a case to protect your cell phone?")

Setting `pad_token_id` to `eos_token_id`:49152 for open-end generation.


[{'generated_text': 'Should you buy a case to protect your cell phone?";\n    public static final String MESSAGE_USAGE = COMMAND_WORD + ": " + MESSAGE_DESCRIPTION + "\\n"\n            + "Parameters: " + PREFIX_CASE_NUMBER + "CASE_NUM\\n" + "Example: "\n            + COMMAND_PREFIX + " " + COMMANDWORD + " 1";\n\n    private final CaseNumber caseNumber;\n\n    public BuyCaseCommand(CaseNumber caseNum) {\n        requireNonNull(caseNum);\n        this.caseNumber = caseNum;\n    }\n\n    @Override\n    public CommandResult execute(Model model) throws CommandException {\n        requireAllNonNull(model);\n\n        if (!model.hasCase(caseNumber)) {\n            throw new CommandException(MESSAGE_CASENOTFOUND);\n        }\n\n        if (model.isCaseInStock(caseName)) {\n\n            model.buyCase(model.getFilteredCaseList().get(0));\n            return new CommandResult(String.format(MESSAGE__BUY_CASE, caseName));\n        } else {\n            return null;\n        }\n    }\n}\n'}]

In [19]:
pipe("Symptoms of fever?")

Setting `pad_token_id` to `eos_token_id`:49152 for open-end generation.


[{'generated_text': 'Symptoms of fever? (yes/no) " +\n                "Please enter yes or no: ");\n        String fever = input.nextLine();\n        if (fever.equals("yes")) {\n            System.out.println("Do you have any other symptoms of cough? (Yes/No) "\n                    + "Please Enter Yes or No: ");\n            String cough = input2.nextLine().toLowerCase();\n            if (cough.equals(coughYes)) {\n                System.err.println(cougYes);\n                SystemOut.print(couYes);\n            } else if (couNo.equals((coughNo))) {\n                //System.out\n                SystemErr.print("Please enter the symptoms you have: ");\n                String symptoms = input3.nextLine() + " " + input4.nextLine();// + "\n                //+ " "\n                String[] symptomsArr = symptoms.split(" ");\n                for (int i = 0; i < symptomsArray.length; i++) {\n                    if (symptomsArray[i].equals(symptomsArr[i])) {\n                        System.e

In [21]:
pipe_dpo("Human:Symptoms of fever? Assistant:")

Setting `pad_token_id` to `eos_token_id`:49152 for open-end generation.


[{'generated_text': 'Human:Symptoms of fever? Assistant: Yes, I have a lot of fear.\nHuman:What is the most common symptoms of the day?\nAssistant:\nAnswer:\nHuman 1:  Sore throat\nHuman2:  Fever\nHuman3:  Cough\nHuman4:  Breathing\nHuman5:  Headache\nHuman6:  Nausea\nHuman7:  Diarrhea\nHuman8:  Loss of taste\nHuman9:  Stress\nHuman10:  Lack of breath\nHuman\nAssistant 1\nAnswer\nHuman'}]

In [20]:
pipe("Does knee brace help with knee pain?")

Setting `pad_token_id` to `eos_token_id`:49152 for open-end generation.


[{'generated_text': 'Does knee brace help with knee pain?\n\n# In[10]:\n\n\n# 1.1.2.1\n# Create a function that takes a string and returns the string with the first letter of each word capitalized.\n\ndef capitalize_first_letter(string):\n    return string.title()\n\n\nprint(capitalize_firstLetter("hello world"))\n\n\ndef test_capitalize_string():\n    assert capitalize_string("hello") == "Hello"\n    assert (\n        capitalize_str("hello", "world") == \'Hello world\'\n    ), "capitalize_str should return the string in the same case as the input"\n    # assert capitalizeString("hello"), "capitalizeString should return a string in same case"\n    return\n\n\ntest_capitalizeString()\n\n'}]

In [23]:
pipe_dpo("Human:Does knee brace help with knee pain? Assistant:")

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:49152 for open-end generation.


[{'generated_text': "Human:Does knee brace help with knee pain? Assistant: Yes.\nHuman:What is the most common way to get knee pressure?\nAssistant:\nAnswer:\nHuman 1:  I can get knees in a straight line.\nAssistant 1a:  The knee can be in the middle of the line.  I think it's a good idea to get the knee in the center of the kneel.\nAnswer 1b:  It's a bit more complicated, but it's not a good thing to get a knee at the center.  It can be a good way to make a kneer.\n\nHuman, I'm not sure.  What can I do to get my knee?\n\nAssistant, I can help you.  You can ask me about the knob, or you can ask for help.  If you want to know more about the knobs, you can read the knock-knock guide.\n"}]

In [24]:
pipe("Do people ever recover from long covid?")

Setting `pad_token_id` to `eos_token_id`:49152 for open-end generation.


[{'generated_text': 'Do people ever recover from long covid?\n\n# In[10]:\n\n\n# 1. Importing the libraries\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport pandas as pd\nimport seaborn as sns\n\nfrom sklearn.model_selection import train_test_split\nfrom tensorflow.keras.models import Sequential\nfrom keras.layers import Dense\nfrom matplotlib import style\n\nstyle.use(\'ggplot\')\n\n\ndef load_data(path):\n    """\n    Load the data from the given path.\n    """\n\n    # Load the dataset\n    df = pd.read_csv(path)\n\n    # Drop the unnecessary columns\n    df.drop([\'Unnamed: 0\'], axis=1, inplace=True)\n\n    return df\n\n\ndf = load_dataset(\'data/covid_data.csv\')\n\ndf.head()\n\n\nX = df.iloc[:, 1:2].values\ny = df[\'death\'].values\n\nX_train, X_test, y_train, y_test = train_val_test(X, y, test_size=0.2, random_state=0)\n\n\nfrom scipy.stats import randint as sp_randint'}]

In [31]:
pipe("What are benfits of Regular exercise?")

Setting `pad_token_id` to `eos_token_id`:49152 for open-end generation.


[{'generated_text': 'What are benfits of Regular exercise?\n\n# In[10]:\n\n\n# Exercise 1\n# Create a function that takes a list of numbers and returns the sum of the numbers.\n\ndef sum_of_numbers(numbers):\n    return sum(numbers)\n\n\nprint(sum_of(numbers))\n\n\ndef add_numbers_to_list(numbers, list):\n    for number in numbers:\n        list.append(number)\n    return list\n\n\nnumbers = [1, 2, 3, 4, 5]\nprint("Original numbers: ", numbers)\nprint("\\nList after adding numbers: ")\nprint_list = add_number_to(numbers=numbers, number=10)\n\nprint()\nprint(\'List after appending numbers: \')\nprint print_list\n\n\n## Exercise 2\n# Write a function called "add_numbers" that takes two lists and returns a new list that contains the elements of the first list plus the elements from the second list.\n# \n# For example, if the first and second lists are [10, 15, 7], then the function should return [11,16,'}]

In [30]:
pipe_dpo("Human: What are benfits of Regular exercise? Assistant:")

Setting `pad_token_id` to `eos_token_id`:49152 for open-end generation.


[{'generated_text': "Human: What are benfits of Regular exercise? Assistant: Regular exercise is a way to keep your body from falling asleep.\nHuman: How do you know that you're not going to be able to sleep?\nAssistant: You can't sleep if you're going to have a regular exercise.\n\nHuman 2:\nAssistant 2\n\nAnswer:  I think you're probably going to need a regular workout.  I'm not sure if you can do that, but I think it's a good idea to have regular workouts."}]